# Transfer Learning In PyTorch
The goal of this assignment is to implement pretrained models in Pytorch to perform classification and test it out on the given data.

In the begining, please import all the package you need. We provide some packages here, which might be helpful when you build your code.

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
#from torchvision.models import MobileNet_V3_Large_Weights
#from torchvision.models import MobileNet_V3_Small_Weights

# Task: Use MobileNet V3 for the mini cifar-10 data
However, the images we've used in CIFAR-10 are easy and the image sizes are small. If we need to do a more complicated classification task with high-resolution images, we may use a more deep network model. Here, we will use the pretrain MobileNet v3 large model for the mini cifar-10 data.

In [ ]:
#traindir = f"data/cifar-10-batches-py/train"
#validdir = f"data/cifar-10-batches-py/val"
traindir = f"data/cifar-10-mini/train"
validdir = f"data/cifar-10-mini/val"

#save_file_name = f'mobilenetv3-transfer.pt'
#checkpoint_path = f'mobilenetv3-transfer.pth'

# Change to fit hardware
batch_size = 8
batch_size

For deep learning, data augmentation is significant for increasing the dataset and improving the performance. We will define crop, rotation, color jitter, and flip before we load images in our network.

In [ ]:
# define transforms
# Image transformations
image_transforms = {
    # Train uses data augmentation
    'train':
        transforms.Compose([
            # You can use transforms.RandomResizedCrop() for crop
            # You can use transforms.RandomRotation() for rotation
            # You can use transforms.ColorJitter() for colorjitter
            # You can use transforms.RandomHorizontalFlip() for flip
            #####################
            ### YOUR CODE HERE###
            #####################
            transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
            transforms.RandomRotation(degrees=15),
            transforms.ColorJitter(),
            transforms.RandomHorizontalFlip(),
            ####################
            ### YOUR CODE END###
            ####################
            transforms.CenterCrop(size=224),  # Image net standards
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Imagenet standards
        ]),
    # Validation does not use augmentation
    'valid':
        transforms.Compose([
            transforms.Resize(size=256),
            transforms.CenterCrop(size=224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
}

Now for each original image, we can obtain several processed images. Now we can load the grocery data here. Since we load all the images from folders, and we need to extract the labels from folder names, the dataloader might be different. Then, we can check all the classes in our new dataset.

In [ ]:
# Datasets from folders

data = {
    'train':
    datasets.ImageFolder(root=traindir, transform=image_transforms['train']),
    'valid':
    #####################
    ### YOUR CODE HERE###
    #####################
    datasets.ImageFolder(root=validdir, transform=image_transforms['valid'])
    ####################
    ### YOUR CODE END###
    ####################
}

# Dataloader iterators, make sure to shuffle
# You can use the same dataloader in HW3
#####################
### YOUR CODE HERE###
#####################
dataloaders = {
    'train': torch.utils.data.DataLoader(data['train'], batch_size=batch_size, shuffle=True,num_workers=10),
    'val': torch.utils.data.DataLoader(data['valid'], batch_size=batch_size, shuffle=True,num_workers=10)
}
####################
### YOUR CODE END###
####################

# Iterate through the dataloader once
trainiter = iter(dataloaders['train'])
validationiter = iter(dataloaders['val'])

categories = []
for d in os.listdir(traindir):
    categories.append(d)

n_classes = len(categories)
n_classes
#print(f'There are {n_classes} different classes.')

Let's take a look at the data augmentation we've defined. After the data augmentation, the image values might be out of the range. Therefore, when we print the images, we may clip the value.

In [ ]:
# take a look at data augmentation

def imshow_tensor(image, title=None):
    """Imshow for Tensor."""

    # Set the color channel as the third dimension
    image = image.numpy().transpose((1, 2, 0))

    # Reverse the preprocessing steps
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean

    # Clip the image pixel values
    # You may use np.clip() to clip the value between 0 to 1.
    #####################
    ### YOUR CODE HERE###
    #####################
    image = np.clip(image, 0, 1)
    ####################
    ### YOUR CODE END###
    ####################
    plt.figure(figsize=(24, 24))

    plt.imshow(image)

    if title is not None:
        plt.title(title)
    plt.pause(0.001)


inputs, classes = next(iter(dataloaders['train']))

out = torchvision.utils.make_grid(inputs)
imshow_tensor(out, title = [data['train'].classes[x] for x in classes])

Here, we may define our model with the pretrianed mobilenet v3 large. For the pretrained model, the output channels might match the original dataset. Therefore, we need to change the last block of network specially for our dataset. Because we use pretrain model, we may freeze all the layers that related to the features learning. Then, our model can only learn the feature classifier with the last block we've defined.

In [ ]:
# Define the network with pretrained models.mobilenet_v3_large

model = models.mobilenet_v3_large(weights = models.MobileNet_V3_Large_Weight.IMAGENET1K_V1)





for param in model.parameters():
    param.requires_grad = False
# print(model)

model.classifier = nn.Sequential(

                      nn.Linear(in_features=960, out_features=1280, bias=True),
                      nn.Hardswish(),
                      nn.Dropout(p=0.2, inplace=True),
                      nn.Linear(in_features=1280, out_features=n_classes, bias=True)


                      #nn.Linear(in_features=576, out_features=1024, bias=True),
                      #nn.Hardswish(),
                      #nn.Dropout(p=0.2, inplace=True),
                      #nn.Linear(in_features=1024, out_features=n_classes, bias=True)

                      )
#model.classifier
#model


total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')


Now we can set up the hyper parameters for our network.

In [ ]:
model.class_to_idx = data['train'].class_to_idx
model.idx_to_class = {
    idx: class_
    for class_, idx in model.class_to_idx.items()
}

list(model.idx_to_class.items())


# Set up your criterion and optimizer
# You can use nn.CrossEntropyLoss() as your critenrion
# You can use optim.Adam() as your optimizer with reasonable momentum

#####################
### YOUR CODE HERE###
#####################
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
####################
### YOUR CODE END###
####################

for p in optimizer.param_groups[0]['params']:
    if p.requires_grad:
        print(p.shape)

Well down! Now we can start our training process here.

In [ ]:
def train(model,
          criterion,
          optimizer,
          train_loader,
          valid_loader,
          save_file_name,
          max_epochs_stop=3,
          n_epochs=10,
          print_every=1):
    """Train a PyTorch Model

    Params
    --------
        model (PyTorch model): model to train
        criterion (PyTorch loss): objective to minimize
        optimizer (PyTorch optimizier): optimizer to compute gradients of model parameters
        train_loader (PyTorch dataloader): training dataloader to iterate through
        valid_loader (PyTorch dataloader): validation dataloader used for early stopping
        save_file_name (str ending in '.pt'): file path to save the model state dict
        max_epochs_stop (int): maximum number of epochs with no improvement in validation loss for early stopping
        n_epochs (int): maximum number of training epochs
        print_every (int): frequency of epochs to print training stats

    Returns
    --------
        model (PyTorch model): trained cnn with best weights
        history (DataFrame): history of train and validation loss and accuracy
    """

    # Early stopping intialization
    epochs_no_improve = 0
    valid_loss_min = np.Inf

    valid_max_acc = 0
    history = []

    # Number of epochs already trained (if using loaded in model weights)
    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.\n')

    overall_start = timer()

    # Main loop
    for epoch in range(n_epochs):

        # keep track of training and validation loss each epoch

        train_loss = 0.0
        valid_loss = 0.0

        train_acc = 0
        valid_acc = 0


        # Set to training

        model.train()

        start = timer()

        # Training loop
        for ii, (data, target) in enumerate(train_loader):

            # Clear gradients
            optimizer.zero_grad()

            # Get your output from your model

            model = model.float()
            output = model(data.float())

            # Loss and backpropagation of gradients

            loss = criterion(output, target.long())
            loss.backward()

            # Update the parameters
            #####################
            ### YOUR CODE HERE###
            #####################
            optimizer.step()
            #####################
            ### YOUR CODE END ###
            #####################

            # Track train loss by multiplying average loss by number of examples in batch

            train_loss += loss.item() * data.size(0)



            # Calculate accuracy by finding max log probability

            _, pred = torch.max(output, dim=1)
            correct_tensor = pred.eq(target.data.view_as(pred))


            # Need to convert correct tensor from int to float to average

            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))


            # Multiply average accuracy times the number of examples in batch

            train_acc += accuracy.item() * data.size(0)


            # Track training progress
            print(
                f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete. {timer() - start:.2f} seconds elapsed in epoch.',
                end='\r')

        # After training loops ends, start validation
        else:
            model.epochs += 1

            # Don't need to keep track of gradients
            with torch.no_grad():

                # Set to evaluation mode

                model.eval()


                # Validation loop
                for data, target in valid_loader:



                    # Forward pass

                    model = model.float()
                    output = model(data.float())


                    # Validation loss

                    loss = criterion(output, target.long())



                    # Multiply average loss times the number of examples in batch

                    valid_loss += loss.item() * data.size(0)


                    # Calculate validation accuracy

                    _, pred = torch.max(output, dim=1)
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))


                    # Multiply average accuracy times the number of examples

                    valid_acc += accuracy.item() * data.size(0)


                # Calculate average losses

                train_loss = train_loss / len(train_loader.dataset)
                valid_loss = valid_loss / len(valid_loader.dataset)


                # Calculate average accuracy

                train_acc = train_acc / len(train_loader.dataset)
                valid_acc = valid_acc / len(valid_loader.dataset)


                history.append([train_loss, valid_loss, train_acc, valid_acc])

                # Print training and validation results
                if (epoch + 1) % print_every == 0:
                    print(
                        f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}'
                    )
                    print(
                        f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%'
                    )

                # Save the model if validation loss decreases
                if valid_loss < valid_loss_min:
                    # Save model
                    torch.save(model.state_dict(), save_file_name)
                    # Track improvement
                    epochs_no_improve = 0
                    valid_loss_min = valid_loss
                    valid_best_acc = valid_acc
                    best_epoch = epoch

                # Otherwise increment count of epochs with no improvement
                else:
                    epochs_no_improve += 1
                    # Trigger early stopping
                    if epochs_no_improve >= max_epochs_stop:
                        print(
                            f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
                        )
                        total_time = timer() - overall_start
                        print(
                            f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.'
                        )

                        # Load the best state dict

                        model.load_state_dict(torch.load(save_file_name))

                        # Attach the optimizer
                        model.optimizer = optimizer

                        # Format history
                        history = pd.DataFrame(
                            history,
                            columns=[
                                'train_loss', 'valid_loss', 'train_acc',
                                'valid_acc'
                            ])
                        return model, history

    # Attach the optimizer
    model.optimizer = optimizer
    # Record overall time and print out stats
    total_time = timer() - overall_start
    print(
        f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_best_acc:.2f}%'
    )
    print(
        f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.'
    )
    # Format history
    history = pd.DataFrame(
        history,
        columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])
    return model, history

In [ ]:
from timeit import default_timer as timer
save_file_name = f'mobilenet_v3_model_best_model.pt'


model, history = train(model,
    criterion,
    optimizer,
    dataloaders['train'],
    dataloaders['val'],
    save_file_name=save_file_name,
    max_epochs_stop=3,
    n_epochs=5,
    print_every=1)

In [ ]:
plt.figure(figsize=(8, 6))
for c in ['train_loss', 'valid_loss']:
    plt.plot(
        history[c], label=c)
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Average Negative Log Likelihood')
plt.title('Training and Validation Losses')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
for c in ['train_acc', 'valid_acc']:
    plt.plot(
        100 * history[c], label=c)
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Average Accuracy')
plt.title('Training and Validation Accuracy')
plt.show()

In [ ]:
# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    npimg = npimg.transpose(1,2,0)

    plt.imshow(npimg)
    plt.show()


dataiter = iter(validationiter)
# get some random training images
# you may use .next() to get the next iteration of validation dataloader
######################
### YOUR CODE HERE ###
######################
images, labels = dataiter.__next__()
#####################
### YOUR CODE END ###
#####################

# Get the prediction of images by using your model.
#####################
### YOUR CODE HERE###
#####################
outputs = model(images)
_, predicted = torch.max(outputs, 1)
#predicted
####################
### YOUR CODE END###
####################

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % categories[labels[j].long()] for j in range(batch_size)))
print('Prediction: ', ' '.join('%5s' % categories[predicted[j].long()] for j in range(batch_size)))